# SQL Fundamentals - Sample Database Setup (SQLite)


## Tables Created:
- **customers** (~100 rows) - Customer information
- **orders** (~300 rows) - Order transactions
- **order_items** (~600 rows) - Individual items within orders
- **products** (~50 rows) - Product catalog
- **employees** (~50 rows) - Employee hierarchy

# setup the connection and data setup

In [35]:
import sqlite3
import random
from datetime import datetime, timedelta
import pandas as pd

# Create SQLite database (file-based, persists automatically)
# Use ':memory:' for in-memory database that doesn't persist
con = sqlite3.connect('sql_practice.db')
cur = con.cursor()

print("SQLite connection established!")
print(f"SQLite version: {sqlite3.sqlite_version}")

SQLite connection established!
SQLite version: 3.37.2


## Sample Data Generation

First, let's define some realistic sample data to populate our tables.

In [36]:
# Sample data pools
FIRST_NAMES = [
    'James', 'Mary', 'John', 'Patricia', 'Robert', 'Jennifer', 'Michael', 'Linda',
    'William', 'Elizabeth', 'David', 'Barbara', 'Richard', 'Susan', 'Joseph', 'Jessica',
    'Thomas', 'Sarah', 'Charles', 'Karen', 'Christopher', 'Lisa', 'Daniel', 'Nancy',
    'Matthew', 'Betty', 'Anthony', 'Margaret', 'Mark', 'Sandra', 'Donald', 'Ashley',
    'Steven', 'Kimberly', 'Paul', 'Emily', 'Andrew', 'Donna', 'Joshua', 'Michelle',
    'Kenneth', 'Dorothy', 'Kevin', 'Carol', 'Brian', 'Amanda', 'George', 'Melissa',
    'Timothy', 'Deborah', 'Ronald', 'Stephanie', 'Edward', 'Rebecca', 'Jason', 'Sharon',
    'Jeffrey', 'Laura', 'Ryan', 'Cynthia', 'Jacob', 'Kathleen', 'Gary', 'Amy',
    'Nicholas', 'Angela', 'Eric', 'Shirley', 'Jonathan', 'Anna', 'Stephen', 'Brenda'
]

LAST_NAMES = [
    'Smith', 'Johnson', 'Williams', 'Brown', 'Jones', 'Garcia', 'Miller', 'Davis',
    'Rodriguez', 'Martinez', 'Hernandez', 'Lopez', 'Gonzalez', 'Wilson', 'Anderson',
    'Thomas', 'Taylor', 'Moore', 'Jackson', 'Martin', 'Lee', 'Perez', 'Thompson',
    'White', 'Harris', 'Sanchez', 'Clark', 'Ramirez', 'Lewis', 'Robinson', 'Walker',
    'Young', 'Allen', 'King', 'Wright', 'Scott', 'Torres', 'Nguyen', 'Hill', 'Flores',
    'Green', 'Adams', 'Nelson', 'Baker', 'Hall', 'Rivera', 'Campbell', 'Mitchell', 'Carter'
]

CITIES = [
    'NYC', 'LA', 'Chicago', 'Houston', 'Phoenix', 'Philadelphia', 'San Antonio',
    'San Diego', 'Dallas', 'San Jose', 'Austin', 'Jacksonville', 'Fort Worth',
    'Columbus', 'Charlotte', 'San Francisco', 'Indianapolis', 'Seattle', 'Denver',
    'Boston', 'Nashville', 'Portland', 'Miami', 'Atlanta', 'Detroit'
]

EMAIL_DOMAINS = ['gmail.com', 'yahoo.com', 'hotmail.com', 'outlook.com', 'icloud.com', 'proton.me']

PRODUCT_CATEGORIES = ['Electronics', 'Clothing', 'Home & Garden', 'Sports', 'Books', 'Toys', 'Beauty', 'Food']

PRODUCTS_BY_CATEGORY = {
    'Electronics': [
        ('Wireless Headphones', 79.99), ('Bluetooth Speaker', 49.99), ('USB-C Hub', 34.99),
        ('Mechanical Keyboard', 129.99), ('Gaming Mouse', 59.99), ('Webcam HD', 89.99),
        ('Portable Charger', 29.99), ('Smart Watch', 199.99), ('Tablet Stand', 24.99)
    ],
    'Clothing': [
        ('Cotton T-Shirt', 19.99), ('Denim Jeans', 49.99), ('Hoodie', 39.99),
        ('Running Shoes', 89.99), ('Winter Jacket', 129.99), ('Baseball Cap', 24.99)
    ],
    'Home & Garden': [
        ('Plant Pot Set', 34.99), ('LED Desk Lamp', 44.99), ('Throw Blanket', 29.99),
        ('Wall Clock', 39.99), ('Picture Frame Set', 24.99), ('Candle Set', 19.99)
    ],
    'Sports': [
        ('Yoga Mat', 29.99), ('Resistance Bands', 19.99), ('Dumbbell Set', 79.99),
        ('Jump Rope', 14.99), ('Water Bottle', 24.99), ('Sports Bag', 44.99)
    ],
    'Books': [
        ('Python Programming', 44.99), ('Data Science Handbook', 54.99), ('SQL Mastery', 39.99),
        ('Machine Learning Guide', 59.99), ('Business Strategy', 29.99), ('Self-Help Book', 19.99)
    ],
    'Toys': [
        ('Building Blocks Set', 34.99), ('Board Game', 29.99), ('Puzzle 1000pc', 19.99),
        ('Remote Control Car', 49.99), ('Art Supply Kit', 24.99)
    ],
    'Beauty': [
        ('Skincare Set', 59.99), ('Hair Dryer', 44.99), ('Makeup Brush Set', 29.99),
        ('Perfume', 79.99), ('Face Mask Pack', 19.99)
    ],
    'Food': [
        ('Coffee Beans 1kg', 24.99), ('Tea Collection', 19.99), ('Chocolate Box', 34.99),
        ('Snack Variety Pack', 29.99), ('Olive Oil Premium', 39.99)
    ]
}

print("Sample data pools loaded!")

Sample data pools loaded!


In [37]:
# Drop tables if they exist (for re-running the notebook)
tables_to_drop = ['order_items', 'orders', 'customers', 'products', 'employees', 'customers_archive', 'temp_table']
for table in tables_to_drop:
    cur.execute(f"DROP TABLE IF EXISTS {table}")

# Create the customers table
cur.execute("""
    CREATE TABLE customers (
        customer_id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        email TEXT UNIQUE NOT NULL,
        city TEXT,
        phone TEXT,
        created_at TEXT DEFAULT CURRENT_TIMESTAMP,
        total_orders INTEGER DEFAULT 0
    )
""")

# Create the products table
cur.execute("""
    CREATE TABLE products (
        product_id INTEGER PRIMARY KEY,
        product_name TEXT NOT NULL,
        category TEXT NOT NULL,
        price REAL NOT NULL,
        stock_quantity INTEGER DEFAULT 100
    )
""")

# Create the orders table
cur.execute("""
    CREATE TABLE orders (
        order_id INTEGER PRIMARY KEY,
        customer_id INTEGER NOT NULL,
        order_date TEXT NOT NULL,
        total_amount REAL NOT NULL,
        status TEXT DEFAULT 'completed',
        FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
    )
""")

# Create the order_items table
cur.execute("""
    CREATE TABLE order_items (
        item_id INTEGER PRIMARY KEY,
        order_id INTEGER NOT NULL,
        product_id INTEGER NOT NULL,
        product_name TEXT NOT NULL,
        quantity INTEGER NOT NULL,
        price REAL NOT NULL,
        FOREIGN KEY (order_id) REFERENCES orders(order_id),
        FOREIGN KEY (product_id) REFERENCES products(product_id)
    )
""")

# Create the employees table (for self-join examples)
cur.execute("""
    CREATE TABLE employees (
        employee_id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        department TEXT,
        manager_id INTEGER,
        hire_date TEXT,
        salary REAL,
        FOREIGN KEY (manager_id) REFERENCES employees(employee_id)
    )
""")

# Create archive table for INSERT examples
cur.execute("""
    CREATE TABLE customers_archive (
        customer_id INTEGER,
        name TEXT,
        email TEXT,
        city TEXT,
        phone TEXT,
        created_at TEXT,
        total_orders INTEGER
    )
""")

# Create temp table for DELETE/TRUNCATE examples
cur.execute("""
    CREATE TABLE temp_table (
        id INTEGER,
        data TEXT
    )
""")

con.commit()
print("All tables created successfully!")

All tables created successfully!


In [38]:
# Set random seed for reproducibility
random.seed(42)

def generate_phone():
    """Generate a random US phone number or None (30% chance of no phone)"""
    if random.random() < 0.3:
        return None
    return f"+1-{random.randint(200,999)}-{random.randint(100,999)}-{random.randint(1000,9999)}"

def generate_date(start_year=2020, end_year=2025):
    """Generate a random date between start_year and end_year"""
    start = datetime(start_year, 1, 1)
    end = datetime(end_year, 12, 31)
    delta = end - start
    random_days = random.randint(0, delta.days)
    return (start + timedelta(days=random_days)).strftime('%Y-%m-%d')

print("Helper functions defined!")

Helper functions defined!


In [39]:
# Generate and insert customers (~100 rows)
customers_data = []
used_emails = set()

for i in range(1, 101):
    first = random.choice(FIRST_NAMES)
    last = random.choice(LAST_NAMES)
    name = f"{first} {last}"

    # Generate unique email
    base_email = f"{first.lower()}.{last.lower()}"
    domain = random.choice(EMAIL_DOMAINS)
    email = f"{base_email}@{domain}"

    # Ensure uniqueness
    counter = 1
    while email in used_emails:
        email = f"{base_email}{counter}@{domain}"
        counter += 1
    used_emails.add(email)

    city = random.choice(CITIES)
    phone = generate_phone()
    created_at = generate_date(2018, 2024)

    customers_data.append((i, name, email, city, phone, created_at, 0))

# Bulk insert customers
cur.executemany("""
    INSERT INTO customers (customer_id, name, email, city, phone, created_at, total_orders)
    VALUES (?, ?, ?, ?, ?, ?, ?)
""", customers_data)
con.commit()

print(f"Inserted {len(customers_data)} customers")

Inserted 100 customers


In [40]:
# Generate and insert products (~50 rows)
products_data = []
product_id = 1

for category, products in PRODUCTS_BY_CATEGORY.items():
    for product_name, base_price in products:
        # Add some price variation
        price = round(base_price * random.uniform(0.9, 1.1), 2)
        stock = random.randint(10, 500)
        products_data.append((product_id, product_name, category, price, stock))
        product_id += 1

cur.executemany("""
    INSERT INTO products (product_id, product_name, category, price, stock_quantity)
    VALUES (?, ?, ?, ?, ?)
""", products_data)
con.commit()

print(f"Inserted {len(products_data)} products")

Inserted 48 products


In [41]:
# Generate orders (~300 rows) and order_items (~600 rows)
orders_data = []
order_items_data = []
order_id = 1
item_id = 1

# Some customers order more frequently than others
for customer_id in range(1, 101):
    # Random number of orders per customer (0-10, weighted toward lower numbers)
    num_orders = random.choices(
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
        weights=[5, 20, 25, 20, 12, 8, 5, 3, 1, 0.5, 0.5]
    )[0]

    for _ in range(num_orders):
        order_date = generate_date(2023, 2025)

        # Each order has 1-5 items
        num_items = random.randint(1, 5)
        order_total = 0

        # Select random products for this order
        selected_products = random.sample(products_data, min(num_items, len(products_data)))

        for prod in selected_products:
            prod_id, prod_name, _, prod_price, _ = prod
            quantity = random.randint(1, 3)
            item_total = prod_price * quantity
            order_total += item_total

            order_items_data.append((item_id, order_id, prod_id, prod_name, quantity, prod_price))
            item_id += 1

        status = random.choices(
            ['completed', 'pending', 'shipped', 'cancelled'],
            weights=[70, 10, 15, 5]
        )[0]

        orders_data.append((order_id, customer_id, order_date, round(order_total, 2), status))
        order_id += 1

# Insert orders
cur.executemany("""
    INSERT INTO orders (order_id, customer_id, order_date, total_amount, status)
    VALUES (?, ?, ?, ?, ?)
""", orders_data)

# Insert order items
cur.executemany("""
    INSERT INTO order_items (item_id, order_id, product_id, product_name, quantity, price)
    VALUES (?, ?, ?, ?, ?, ?)
""", order_items_data)

con.commit()

print(f"Inserted {len(orders_data)} orders")
print(f"Inserted {len(order_items_data)} order items")

Inserted 255 orders
Inserted 759 order items


In [42]:
# Update customer total_orders count
cur.execute("""
    UPDATE customers
    SET total_orders = (
        SELECT COUNT(*)
        FROM orders
        WHERE orders.customer_id = customers.customer_id
    )
""")
con.commit()

print("Updated customer order counts!")

Updated customer order counts!


In [43]:
# Generate employees (~50 rows) with manager hierarchy
DEPARTMENTS = ['Engineering', 'Sales', 'Marketing', 'HR', 'Finance', 'Operations', 'Customer Support']

employees_data = []

# Create CEO (no manager)
employees_data.append((1, 'Sarah Mitchell', 'Executive', None, '2015-03-15', 250000))

# Create department heads (report to CEO)
dept_heads = []
emp_id = 2
for dept in DEPARTMENTS:
    name = f"{random.choice(FIRST_NAMES)} {random.choice(LAST_NAMES)}"
    hire_date = generate_date(2016, 2020)
    salary = round(random.uniform(120000, 180000), 2)
    employees_data.append((emp_id, name, dept, 1, hire_date, salary))
    dept_heads.append((emp_id, dept))
    emp_id += 1

# Create regular employees (report to department heads)
while emp_id <= 50:
    manager_id, dept = random.choice(dept_heads)
    name = f"{random.choice(FIRST_NAMES)} {random.choice(LAST_NAMES)}"
    hire_date = generate_date(2018, 2024)
    salary = round(random.uniform(50000, 100000), 2)
    employees_data.append((emp_id, name, dept, manager_id, hire_date, salary))
    emp_id += 1

cur.executemany("""
    INSERT INTO employees (employee_id, name, department, manager_id, hire_date, salary)
    VALUES (?, ?, ?, ?, ?, ?)
""", employees_data)
con.commit()

print(f"Inserted {len(employees_data)} employees")

Inserted 50 employees


In [44]:
# Add some data to temp_table for DELETE examples
temp_data = [(i, f"Sample data row {i}") for i in range(1, 21)]
cur.executemany("INSERT INTO temp_table VALUES (?, ?)", temp_data)
con.commit()
print(f"Inserted {len(temp_data)} rows into temp_table")

Inserted 20 rows into temp_table


In [45]:
# Helper function to run queries and display as DataFrame
def query(sql):
    """Execute SQL and return as pandas DataFrame"""
    return pd.read_sql_query(sql, con)

print("Query helper function ready!")

Query helper function ready!


# QUERY DATA

In [46]:
# Check row counts for all tables
tables = ['customers', 'products', 'orders', 'order_items', 'employees', 'temp_table']

print("=" * 40)
print("TABLE ROW COUNTS")
print("=" * 40)

for table in tables:
    count = cur.execute(f"SELECT COUNT(*) FROM {table}").fetchone()[0]
    print(f"{table:20} {count:>6} rows")

print("=" * 40)

TABLE ROW COUNTS
customers               100 rows
products                 48 rows
orders                  255 rows
order_items             759 rows
employees                50 rows
temp_table               20 rows


## GROUP BY

In [135]:
query(

    """

    SELECT

    customer_id,
    COUNT(*) AS num_orders,
    SUM(total_amount) AS total_spent,
    AVG(total_amount) AS avg_order_value
    FROM orders
    GROUP BY customer_id

    """
)

,customer_id,num_orders,total_spent,avg_order_value
0,1,1,249.92,249.920000
1,3,2,510.97,255.485000
2,4,1,410.84,410.840000
3,5,6,1219.53,203.255000
4,6,3,965.03,321.676667
...,...,...,...,...
92,96,4,1015.59,253.897500
93,97,3,1212.95,404.316667
94,98,2,391.58,195.790000
95,99,4,2279.74,569.935000


In [127]:
query(

    """

    SELECT
    customer_id,
    COUNT(*) AS num_orders,
    SUM(total_amount) AS total_spent,
    AVG(total_amount) AS avg_order_value
    FROM orders
    GROUP BY customer_id,status

    """
)

,customer_id,num_orders,total_spent,avg_order_value
0,1,1,249.92,249.920000
1,3,2,510.97,255.485000
2,4,1,410.84,410.840000
3,5,1,168.71,168.710000
4,5,4,799.99,199.997500
...,...,...,...,...
154,98,2,391.58,195.790000
155,99,3,2169.40,723.133333
156,99,1,110.34,110.340000
157,100,1,680.13,680.130000


In [141]:

query(

    """

    SELECT
    customer_id,
    COUNT(*) AS num_orders,
    SUM(total_amount) AS total_spent,
    AVG(total_amount) AS avg_order_value
    FROM orders
    GROUP BY customer_id
    HAVING avg_order_value > 300

    """

    )

,customer_id,num_orders,total_spent,avg_order_value
0,4,1,410.84,410.840000
1,6,3,965.03,321.676667
2,9,4,1204.83,301.207500
3,17,2,1050.63,525.315000
4,24,2,853.88,426.940000
5,26,1,864.05,864.050000
6,33,1,316.04,316.040000
7,36,4,1229.44,307.360000
8,37,2,678.19,339.095000
9,39,2,783.50,391.750000


In [48]:
query(
   """
   SELECT
    category,
    COUNT(*) AS product_count,
    AVG(price) AS avg_price,
    MIN(price) AS min_price,
    MAX(price) AS max_price
    FROM products
    GROUP BY category
   """
)

,category,product_count,avg_price,min_price,max_price
0,Beauty,5,47.114000,19.79,84.40
1,Books,6,42.856667,18.45,65.58
2,Clothing,6,59.556667,19.81,141.71
3,Electronics,9,78.987778,24.14,219.85
4,Food,5,29.766000,21.95,36.78
5,Home & Garden,6,33.421667,19.43,47.01
6,Sports,6,36.318333,14.37,87.58
7,Toys,5,32.014000,18.21,53.10


In [50]:
query(
    """
    SELECT
    strftime('%Y', order_date) AS year,
    strftime('%m', order_date) AS month,
    COUNT(*) AS num_orders,
    SUM(total_amount) AS monthly_revenue
    FROM orders
    GROUP BY  strftime('%Y', order_date), strftime('%m', order_date)
    ORDER BY year, month
    """
)

,year,month,num_orders,monthly_revenue
0,2023,01,2,123.33
1,2023,02,8,3192.07
2,2023,03,10,3056.45
3,2023,04,7,1176.57
4,2023,05,7,2421.43
5,2023,06,9,3127.79
6,2023,07,7,1698.30
7,2023,08,7,1938.13
8,2023,09,5,2271.20
9,2023,10,9,2255.63


## HAVING

In [51]:
query(
    """

  SELECT
      customer_id,
      COUNT(*) AS num_orders
  FROM orders
  GROUP BY customer_id
  HAVING COUNT(*) > 5

"""
)

,customer_id,num_orders
0,5,6
1,7,6
2,19,7
3,46,6
4,52,6
5,64,7


In [52]:
query(

    """

  SELECT
      category,
      AVG(price) AS avg_price
  FROM products
  GROUP BY category
  HAVING AVG(price) > 50

"""
)




,category,avg_price
0,Clothing,59.556667
1,Electronics,78.987778


In [ ]:
()

In [53]:
query(
    """
    SELECT
    customer_id,
    COUNT(*) AS num_orders,
    SUM(total_amount) AS total_spent
    FROM orders
    WHERE order_date >= '2025-01-01'
    GROUP BY customer_id
    HAVING SUM(total_amount) > 1000
    """
)

,customer_id,num_orders,total_spent
0,7,4,1155.35
1,52,1,1106.93
2,53,2,1097.04
3,68,3,1092.28
4,99,2,1011.21


## JOINS

In [146]:

query(
    """
    SELECT
      o.*
    FROM customers c
    JOIN orders o ON c.customer_id = o.customer_id


    """
)

,order_id,customer_id,order_date,total_amount,status
0,1,1,2024-03-08,249.92,completed
1,2,3,2025-08-20,274.75,shipped
2,3,3,2023-05-26,236.22,shipped
3,4,4,2023-08-24,410.84,shipped
4,5,5,2025-06-18,118.53,completed
...,...,...,...,...,...
250,251,99,2025-05-20,110.34,pending
251,252,99,2024-09-01,105.24,completed
252,253,99,2023-06-28,1163.29,completed
253,254,100,2023-10-13,680.13,completed


In [142]:
#inner join

query(
    """
    SELECT
      c.customer_id,
      c.name,
      o.order_id,
      o.total_amount
    FROM customers c
    JOIN orders o ON c.customer_id = o.customer_id


    """
)



,customer_id,name,order_id,total_amount
0,1,Joseph Johnson,1,249.92
1,3,Robert Johnson,2,274.75
2,3,Robert Johnson,3,236.22
3,4,Patricia Scott,4,410.84
4,5,Emily Smith,5,118.53
...,...,...,...,...
250,99,Kimberly Perez,251,110.34
251,99,Kimberly Perez,252,105.24
252,99,Kimberly Perez,253,1163.29
253,100,Matthew Garcia,254,680.13


In [149]:
query(
    """

    SELECT
    c.customer_id,
    c.name,
    o.customer_id,
    o.order_id,
    o.total_amount
  FROM customers c
  LEFT JOIN orders o ON c.customer_id = o.customer_id;
    """
)

,customer_id,name,customer_id,order_id,total_amount
0,1,Joseph Johnson,1.0,1.0,249.92
1,2,Susan Baker,NaN,NaN,NaN
2,3,Robert Johnson,3.0,3.0,236.22
3,3,Robert Johnson,3.0,2.0,274.75
4,4,Patricia Scott,4.0,4.0,410.84
...,...,...,...,...,...
253,99,Kimberly Perez,99.0,251.0,110.34
254,99,Kimberly Perez,99.0,250.0,900.87
255,99,Kimberly Perez,99.0,253.0,1163.29
256,100,Matthew Garcia,100.0,255.0,361.73


**Task:** Find all products that have never been ordered.

In [124]:
# query(
#     """

#     """
# )

In [150]:
#right joins
query(
    """
    SELECT
    c.customer_id,
    c.name,
    o.order_id,
    o.total_amount
    FROM orders o
    RIGHT JOIN customers c ON c.customer_id = o.customer_id
    """
)





DatabaseError: Execution failed on sql '  
    SELECT 
    c.customer_id,
    c.name,
    o.order_id,
    o.total_amount
    FROM orders o
    RIGHT JOIN customers c ON c.customer_id = o.customer_id
    ': RIGHT and FULL OUTER JOINs are not currently supported

In [68]:
#full outer joins , sqllite does not support full outer joins :D , think what can be the workaround









In [71]:
#CROSS JOINS
query(


"""
  SELECT c.name, p.product_name
  FROM customers c
  CROSS JOIN products p;
"""
)





,name,product_name
0,Joseph Johnson,Wireless Headphones
1,Joseph Johnson,Bluetooth Speaker
2,Joseph Johnson,USB-C Hub
3,Joseph Johnson,Mechanical Keyboard
4,Joseph Johnson,Gaming Mouse
...,...,...
4795,Matthew Garcia,Coffee Beans 1kg
4796,Matthew Garcia,Tea Collection
4797,Matthew Garcia,Chocolate Box
4798,Matthew Garcia,Snack Variety Pack


In [76]:
query("""SELECT * from orders """)

,order_id,customer_id,order_date,total_amount,status
0,1,1,2024-03-08,249.92,completed
1,2,3,2025-08-20,274.75,shipped
2,3,3,2023-05-26,236.22,shipped
3,4,4,2023-08-24,410.84,shipped
4,5,5,2025-06-18,118.53,completed
...,...,...,...,...,...
250,251,99,2025-05-20,110.34,pending
251,252,99,2024-09-01,105.24,completed
252,253,99,2023-06-28,1163.29,completed
253,254,100,2023-10-13,680.13,completed


In [90]:
query(

      """
      SELECT
        c.name AS customer_name,
        o.order_id,
        o.order_date,
        p.product_name,
        oi.quantity,
        p.price,
        (oi.quantity * p.price) AS line_total
      FROM customers c
      INNER JOIN orders o ON c.customer_id = o.customer_id
      INNER JOIN order_items oi ON o.order_id = oi.order_id
      INNER JOIN products p ON oi.product_id = p.product_id
      ORDER BY o.order_date, o.order_id;
      """

)

,customer_name,order_id,order_date,product_name,quantity,price,line_total
0,Elizabeth Robinson,181,2023-01-08,Picture Frame Set,2,23.46,46.92
1,Edward Perez,203,2023-01-19,Self-Help Book,2,18.45,36.90
2,Edward Perez,203,2023-01-19,SQL Mastery,1,39.51,39.51
3,Richard Jones,106,2023-02-03,Remote Control Car,2,53.10,106.20
4,Richard Jones,106,2023-02-03,Plant Pot Set,1,36.66,36.66
...,...,...,...,...,...,...,...
754,Stephanie Thomas,182,2025-12-25,Resistance Bands,3,18.50,55.50
755,Donna Garcia,25,2025-12-28,Candle Set,3,19.43,58.29
756,Donna Garcia,25,2025-12-28,Makeup Brush Set,1,31.73,31.73
757,Donna Garcia,25,2025-12-28,Chocolate Box,3,35.47,106.41


In [88]:
#self joins
query(

    """
    SELECT
    o.order_date AS order_date,
    m.status AS status
    FROM orders o
    LEFT JOIN orders m ON o.order_date = date(m.order_date, '+1 day')
  WHERE o.order_date IS NOT NULL
"""
)




,order_date,status
0,2024-03-08,None
1,2025-08-20,None
2,2023-05-26,None
3,2023-08-24,pending
4,2025-06-18,None
...,...,...
256,2025-05-20,None
257,2024-09-01,None
258,2023-06-28,None
259,2023-10-13,None



**Task:** Classify customers into spending tiers:
- **VIP**: Total spending > $1000
- **Regular**: Total spending $500-$1000  
- **Low**: Total spending < $500
- **Inactive**: No orders

Show customer name, total spent, and tier. Order by total spent descending.[link text](https://)

**Task:** Find customers who have placed orders in at least 3 different months.

In [89]:
#subqueries





In [ ]:
#CTE



## WINDOW FUNCTIONS

In [97]:
query(

     """
     SELECT SUM(total_amount) FROM orders
     """
)

,SUM(total_amount)
0,72038.28


In [98]:
query(

     """
     SELECT
     customer_id,
     status,
     SUM(total_amount) OVER () AS revenue
     FROM orders
     """
)


,customer_id,status,revenue
0,1,completed,72038.28
1,3,shipped,72038.28
2,3,shipped,72038.28
3,4,shipped,72038.28
4,5,completed,72038.28
...,...,...,...
250,99,pending,72038.28
251,99,completed,72038.28
252,99,completed,72038.28
253,100,completed,72038.28


In [99]:
query(
        """
     SELECT
     customer_id,
     status,
     SUM(total_amount) OVER (order by status) AS revenue
     FROM orders
     """

)

,customer_id,status,revenue
0,5,cancelled,5893.59
1,6,cancelled,5893.59
2,7,cancelled,5893.59
3,9,cancelled,5893.59
4,14,cancelled,5893.59
...,...,...,...
250,89,shipped,72038.28
251,95,shipped,72038.28
252,96,shipped,72038.28
253,97,shipped,72038.28


In [100]:
# -- GROUP BY: Collapses rows
query(

    """
    SELECT customer_id, SUM(total_amount) AS total
    FROM orders
    GROUP BY customer_id;
    """
)


,customer_id,total
0,1,249.92
1,3,510.97
2,4,410.84
3,5,1219.53
4,6,965.03
...,...,...
92,96,1015.59
93,97,1212.95
94,98,391.58
95,99,2279.74


In [152]:
# -- Window Function: Keeps all rows
query(
  """SELECT
    customer_id,
    order_id,
    total_amount,
    status,
    SUM(total_amount) OVER (PARTITION BY status) AS status_total
  FROM orders;"""

)


# -- Result: All rows preserved, with running calculation added

,customer_id,order_id,total_amount,status,status_total
0,5,9,168.71,cancelled,5893.59
1,6,11,267.35,cancelled,5893.59
2,7,19,388.08,cancelled,5893.59
3,9,23,450.82,cancelled,5893.59
4,14,33,386.75,cancelled,5893.59
...,...,...,...,...,...
250,89,225,343.66,shipped,9659.20
251,95,237,228.58,shipped,9659.20
252,96,241,366.29,shipped,9659.20
253,97,245,520.99,shipped,9659.20


In [122]:
query("""select * from order_items""")

,item_id,order_id,product_id,product_name,quantity,price
0,1,1,43,Face Mask Pack,1,19.79
1,2,1,6,Webcam HD,1,87.63
2,3,1,11,Denim Jeans,3,47.50
3,4,2,3,USB-C Hub,3,32.14
4,5,2,15,Baseball Cap,2,23.42
...,...,...,...,...,...,...
754,755,254,14,Winter Jacket,3,141.71
755,756,254,4,Mechanical Keyboard,2,127.50
756,757,255,46,Chocolate Box,2,35.47
757,758,255,19,Wall Clock,3,43.83


In [108]:
#raw numbner

query(
   """ SELECT
    order_id,
    customer_id,
    total_amount,
    ROW_NUMBER() OVER (ORDER BY total_amount DESC) AS overall_rank
FROM orders;

"""
)

,order_id,customer_id,total_amount,overall_rank
0,253,99,1163.29,1
1,123,52,1106.93,2
2,175,68,955.63,3
3,228,91,901.99,4
4,250,99,900.87,5
...,...,...,...,...
250,209,83,31.73,251
251,88,38,27.08,252
252,15,7,23.42,253
253,40,16,23.12,254


In [109]:
query(
   """ SELECT
    order_id,
    customer_id,
    total_amount,
    rank() OVER (ORDER BY total_amount DESC) AS overall_rank
FROM orders;

"""
)

,order_id,customer_id,total_amount,overall_rank
0,253,99,1163.29,1
1,123,52,1106.93,2
2,175,68,955.63,3
3,228,91,901.99,4
4,250,99,900.87,5
...,...,...,...,...
250,209,83,31.73,251
251,88,38,27.08,252
252,15,7,23.42,253
253,40,16,23.12,254


**Task:** Calculate running total of order amounts for each customer, resetting at the start of each year.


# Tasks

**Task:** Calculate the month-over-month revenue growth rate as a percentage. Show month, revenue, previous month revenue, and growth rate. Use orders

**Task:** Find all products that have never been ordered.

## Reconnecting to the Database

The database is saved as `sql_practice.db`. To reconnect later:

In [32]:
# To reconnect to the database later:
# con = sqlite3.connect('sql_practice.db')
# cur = con.cursor()

# Or use pandas directly:
# df = pd.read_sql_query("SELECT * FROM customers", sqlite3.connect('sql_practice.db'))

In [33]:
# Close connection when done (optional, good practice)
con.close()
print("Database 'sql_practice.db' is ready for use!")

Database 'sql_practice.db' is ready for use!


## Quick Reference: Table Schemas

```
customers
├── customer_id (INTEGER, PK)
├── name (TEXT)
├── email (TEXT, UNIQUE)
├── city (TEXT)
├── phone (TEXT, nullable)
├── created_at (TEXT)
└── total_orders (INTEGER)

products
├── product_id (INTEGER, PK)
├── product_name (TEXT)
├── category (TEXT)
├── price (REAL)
└── stock_quantity (INTEGER)

orders
├── order_id (INTEGER, PK)
├── customer_id (INTEGER, FK → customers)
├── order_date (TEXT)
├── total_amount (REAL)
└── status (TEXT)

order_items
├── item_id (INTEGER, PK)
├── order_id (INTEGER, FK → orders)
├── product_id (INTEGER, FK → products)
├── product_name (TEXT)
├── quantity (INTEGER)
└── price (REAL)

employees
├── employee_id (INTEGER, PK)
├── name (TEXT)
├── department (TEXT)
├── manager_id (INTEGER, FK → employees, nullable)
├── hire_date (TEXT)
└── salary (REAL)
```